# Mutation Pipeline

In [ ]:
fr

In [ ]:
from __future__ import print_function
import os.path
import pandas as pd
import gzip
import sys
import numpy as np

sys.path.insert(0, '..')

from src.CCLE_postp_function import *
from JKBio import terra
from JKBio.utils import helper as h
from JKBio.google import gcp
from gsheets import Sheets
from taigapy import TaigaClient
import dalmatian as dm
from genepy.google.google_sheet import dfToSheet

from sklearn.manifold import TSNE
from sklearn.neighbors import KNeighborsClassifier
from scipy.stats import pearsonr,spearmanr

from bokeh.plotting import *
from bokeh.models import HoverTool
from collections import OrderedDict
from IPython.display import Image,display

%load_ext autoreload
%autoreload 2
%load_ext rpy2.ipython
tc = TaigaClient()
output_notebook()

my_id = '~/.client_secret.json'
mystorage_id = "~/.storage.json"
# do the first steps of https://medium.com/craftsmenltd/from-csv-to-google-sheet-using-python-ef097cb014f9
creds = '../.credentials.json'

sheets = Sheets.from_files(my_id, mystorage_id)
replace = {'T': 'Tumor', 'N': 'Normal', 'm': 'Unknown', 'L': 'Unknown'}

## boot up

we are instanciating all the parameters needed for this pipeline to run

In [ ]:
samplesetname = "21Q1"

workspace1="broad-genomics-delivery/Getz_IBM_CellLines_Exomes"
workspace2="broad-firecloud-ccle/CCLE_DepMap_WES"
workspace3="broad-genomics-delivery/CCLE_DepMap_WES"

workspace6="terra-broad-cancer-prod/CCLE_DepMap_WES"
workspace7="terra-broad-cancer-prod/Getz_IBM_CellLines_Exomes"
refworkspace="broad-firecloud-ccle/DepMap_Mutation_Calling_CGA_pipeline"

rnaworkspace="broad-firecloud-ccle/DepMap_hg38_RNAseq"

source1="ibm"
source2="ccle"
source3="ccle"
source6="ccle"
source7="ibm"

refsheet_url = "https://docs.google.com/spreadsheets/d/1Pgb5fIClGnErEqzxpU7qqX6ULpGTDjvzWwDN8XUJKIY"
refsheet_id = "555466897"
sheeturl = "https://docs.google.com/spreadsheets/d/115TUgA1t_mD32SnWAGpW9OKmJ2W5WYAOs3SuSdedpX4"
extract_to_change = {'from_arxspan_id': 'participant'}

MutationWESmethods=[
    "broadinstitute_gtex/samtofastq_v1-0_BETA/6",
    "broadinstitute_gtex/star_v1-0_BETA/7",
    "broadinstitute_gtex/rsem_v1-0_BETA/6",
    "jkobject/rsem_aggregate_results/5",
    "jkobject/rnaseq-germline-snps-indels/7",
    "broadinstitute_gtex/rnaseqc2_v1-0_BETA/2",
    "gkugener/STAR_fusion/17",
    "jkobject/aggregate_vcfs/22",
    "gkugener/Aggregate_files_set/2",
]

maxage='2020-09-10'

In [ ]:
release = samplesetname

ccle_refsamples = sheets.get(refsheet_url).sheets[0].to_frame(index_col=0)
#wm1 = dm.WorkspaceManager(workspace1)
#wm2 = dm.WorkspaceManager(workspace2)
#wm3 = dm.WorkspaceManager(workspace3)

wm6 = dm.WorkspaceManager(workspace6)
wm7 = dm.WorkspaceManager(workspace6)

refwm = dm.WorkspaceManager(refworkspace)

## Adding new data

We are looking for new samples in a range of workspaces.

They are quite messy and might contains duplicates, contain broken file paths...

- We are thus looking at the bam files one by one and comparing them with our own bams. 
- We remove broken files, duplicates and add new version of a cell line's bam if we find some.

In [ ]:
ccle_refsamples

In [ ]:
# we will be missing "primary disease","sm_id", "cellosaurus_id", "gender, "age", "primary_site", "primary_disease", "subtype", "subsubtype", "origin", "comments"
#when SMid: match== 
samples, pairs, noarxspan = GetNewCellLinesFromWorkspaces(refworkspace, stype='wes', maxage=maxage, refurl=refsheet_url, wmfroms = [workspace6, workspace7], sources=[source6,source7], match=['ACH-','CDS-'], participantslicepos=10, accept_unknowntypes=True, extract=extract_to_change, recomputedate=True)

In [ ]:
samples

In [ ]:
noarxspan

### finding back arxspan

In [ ]:
noarxspan = retrieveFromCellLineName(noarxspan, ccle_refsamples, datatype='wes', depmappvlink = "https://docs.google.com/spreadsheets/d/1uqCOos-T9EMQU7y2ZUw4Nm84opU5fIT1y7jet1vnScE", extract=extract_to_change)

In [ ]:
#assess any potential issues
set(noarxspan.arxspan_id) & set(samples.arxspan_id)

In [ ]:
noarxspan.arxspan_id.tolist()

In [ ]:
samples = pd.concat([samples, noarxspan[noarxspan.arxspan_id!='0']], sort=False)

In [ ]:
noarxspan = noarxspan[noarxspan.arxspan_id=='0']

In [ ]:
samples = assessAllSamples(samples, ccle_refsamples, stype='wes', rename={}, extract={})

In [ ]:
samples

In [ ]:
#TODO: manage the match normals in noarxspan samples

## getting the addtional data and writing it here in the right order 'as shown above'
- use the stripped_cell_line_name to find the samples on https://docs.google.com/spreadsheets/d/1uqCOos-T9EMQU7y2ZUw4Nm84opU5fIT1y7jet1vnScE/edit#gid=356471436. 
- Make sure that we don't have duplicate cell lines in there. Otherwise, use the duplicate renaming function
- copy Primary Site, Primary Disease, Subtype, Comments, Disease Sub-subtype, if they exist. (sometimes subtype and subsubtype are the same.. don't use subsubtype then.
- look for the cell line in cellosaurus, you might need to use one of the aliases given in master depmap pv..
- copy  cellosaurus_id gender age info or write 'U' if they don't exist. 'can be a number or {Embryonic, Children, Adult, Fetus, U} 
- check that it does not say this cell line is not a duplicate from another cell line
- check that if it says this cell line is derived/children/father/samepatient from other cell lines, and that if we have any of the other cell lines, that the patient id is changed to be the same one for all (be sure that you are updating everywhere these patient ids are used)

In [ ]:
# If I have a previous samples I can update unknown data directly
samples, notfound = updateFromTracker(samples, ccle_refsamples)

In [ ]:
noarxspan

In [ ]:
noarxspan.sort_values(by = 'stripped_cell_line_name').to_csv('temp/noarxspan_wes_' + release + '.csv')
samples.loc[notfound].to_csv('temp/notfound_wes_'+release+'.csv')
samples.to_csv('temp/new_wes_'+release+'.csv')

In [ ]:
samples.loc[notfound]

In [ ]:
toupdate = {
"primary_disease":['Bone Cancer'],
"cellosaurus_id":[''],
"age":['C'],
"primary_site":['bone'],
"subtype":["Ewings Sarcoma"],
"subsubtype":[""],
"origin":["bone"],
"parent_cell_line":[""],
"matched_normal":[""],
"comments":["(PCR) EWS/FLI"],
"sex":["U"],
"mediatype":["Adherent, CM2:051117: 100.0 %"],
"condition":[""],
'stripped_cell_line_name':['CCLFPEDS0010T'],
"participant_id":['PT-hrwIWtZC']
}
pd.DataFrame(toupdate)

In [ ]:
# updating..
for k, v in toupdate.items():
    samples.loc[notfound,k] =v

In [ ]:
samples['baits'] = 'ice'

In [ ]:
# uploading to our bucket (now a new function)
terra.changeToBucket(samples,'gs://cclebams/wes/', name_col= "index" , values=['internal_bam_filepath','internal_bai_filepath'], filetypes=['bam', 'bai'], catchdup=True, test=False)

In [ ]:
ccle_refsamples = sheets.get(refsheet_url).sheets[0].to_frame(index_col=0)


In [ ]:
#setting the right version TODO: func
names=[]
subccle_refsamples = ccle_refsamples[ccle_refsamples['datatype'] == "wes"]
for k, val in samples.iterrows():
    val = val["arxspan_id"]
    names.append(val)
    samples.loc[k, 'version'] = len(subccle_refsamples[subccle_refsamples['arxspan_id'] == val]) + names.count(val)
samples['version'] = samples['version'].astype(int)

In [ ]:
ccle_refsamples = ccle_refsamples.append(samples, sort=False)

In [ ]:
dfToSheet(ccle_refsamples,'ccle sample tracker', secret=creds)

In [ ]:
pairs = setupPairsFromSamples(samples, subccle_refsamples, extract={'patient_id':'participant_id'})

In [ ]:
#uploading new samples to mut
refwm = refwm.disable_hound()
refwm.upload_samples(samples)
refwm.upload_entities('pairs', pairs)
refwm.update_pair_set(pair_set_id=samplesetname,pair_ids=pairs.index)
sam = refwm.get_samples()

pair = refwm.get_pairs()
refwm.update_pair_set(pair_set_id='all',pair_ids=pair.index)
refwm.update_pair_set(pair_set_id='all_agilent',pair_ids=pair[pair["case_sample"].isin(sam[sam['baits']=="AGILENT"].index.tolist())].index)
refwm.update_pair_set(pair_set_id='all_ice',pair_ids=pair[pair["case_sample"].isin([i for i in sam[(sam['baits'] == "ICE") |(sam['baits'].isna())].index.tolist() if i != 'nan'])].index)
#creating a sample set
refwm.update_sample_set(sample_set_id=samplesetname, sample_ids=samples.index)
refwm.update_sample_set(sample_set_id='all', sample_ids=[i for i in sam.index.tolist() if i!='nan'])
refwm.update_sample_set(sample_set_id='all_agilent', sample_ids = sam[sam['baits'] == "AGILENT"].index.tolist())
refwm.update_sample_set(sample_set_id='all_ice', sample_ids=[i for i in sam[(sam['baits'] == "ICE") |(sam['baits'].isna())].index.tolist() if i != 'nan'])

In [ ]:
#and CN
cnwm = dm.WorkspaceManager('broad-firecloud-ccle/DepMap_WES_CN_hg38')
cnwm = cnwm.disable_hound()
cnwm.upload_samples(samples)
cnwm.upload_entities('pairs', pairs)
cnwm.update_pair_set(pair_set_id=samplesetname,pair_ids=pairs.index)
sam = cnwm.get_samples()

pair = cnwm.get_pairs()
cnwm.update_pair_set(pair_set_id='all',pair_ids=pair.index)
cnwm.update_pair_set(pair_set_id='all_agilent',pair_ids=pair[pair["case_sample"].isin(sam[sam['baits']=="AGILENT"].index.tolist())].index)
cnwm.update_pair_set(pair_set_id='all_ice',pair_ids=pair[pair["case_sample"].isin([i for i in sam[(sam['baits'] == "ICE") |(sam['baits'].isna())].index.tolist() if i != 'nan'])].index)
#creating a sample set
cnwm.update_sample_set(sample_set_id=samplesetname, sample_ids=samples.index)
cnwm.update_sample_set(sample_set_id='all', sample_ids=[i for i in sam.index.tolist() if i!='nan'])
cnwm.update_sample_set(sample_set_id='all_agilent', sample_ids = sam[sam['baits'] == "AGILENT"].index.tolist())
cnwm.update_sample_set(sample_set_id='all_ice', sample_ids=[i for i in sam[(sam['baits'] == "ICE") |(sam['baits'].isna())].index.tolist() if i != 'nan'])

# run the pipeline

We are using Dalmatian to send request to Terra, we are running a set of 5 functions To generate the mutation dataset:

*   For new samples in DepMap, run the ICE version of this task. CCLE2 samples used Agilent targets, so this pipeline should be used instead. The pipelines are identical in terms of their outputs, but the proper targets, baits, and pseudo normal should be used based on how the samples were sequenced.

    **ICE_CGA_Production_Analysis_Pipeline_Cell_Lines_copy** (cclf/CGA_Production_Analysis_Pipeline_Cell_Lines_debuggingSnapshot ID: 22) OR


    **AGILENT_CGA_Production_Analysis_Pipeline_Cell_Lines** (cclf/CGA_Production_Anablysis_Pipeline_Cell_Lines_debuggingSnapshot ID: 22)

*   **common_variant_filter** (breardon/common_variant_filterSnapshot ID: 3)
*   **filterMAF_on_CGA_pipeline** (gkugener/filterMAF_on_CGA_pipelineSnapshot ID: 8)
*   **aggregateMAFs_selectFields** (ccle_mg/aggregateMAFs_selectFieldsSnapshot ID: 1)

This outputs to be downloaded will be saved in the sample set that was run. The output we use for the release is:


*   **passedCGA_filteredMAF_aggregated** 

There are several other tasks in this workspace. In brief:



*   **CGA_Production_Analysis_Pipeline_Cell_Lines** (lelagina/CGA_Production_Analysis_Pipeline_Cell_LinesSnapshot ID: 12). This task is the same as the ICE and AGILENT prefixed version above, except that it relied on pulling the baits and targets to use from the metadata stored for the samples. Having AGILENT and ICE versions specified made the uploading and running process easier.
*   **SANGER_CGA_Production_Analysis_Pipeline_Cell_Lines** (cclf/CGA_Production_Analysis_Pipeline_Cell_Lines_debuggingSnapshot ID: 22). This task was trying to run the CGA pipeline on the Sanger WES data, using a Sanger pseudo normal. In its current implementation, this task fails to complete for the samples.
*   **UNFILTERED_aggregateMAFs_selectFields** (ccle_mg/aggregateMAFs_selectFieldsSnapshot ID: 1). Aggregates the MAF outputted by the CGA cell line pipeline prior to the common variant filter and germline filtering tasks. This can give us insight to which mutations are getting filtered out when. We may want to potentially include this MAF in the release so people can see why certain mutations of interest may be getting filtered out.
*   WES_DM_Mutation_Calling_Pipeline_(standard |expensive) (gkugener/WES_DM_Mutation_Calling_PipelineSnapshot ID: 2). This was a previous mutation calling pipeline implemented for CCLE. We do not use this pipeline any more as the CGA pipeline looks better.
*   aggregate_filterMAF_CGA (CCLE/aggregate_filterMAF_CGASnapshot ID: 1). An aggregation MAF task that we used in the past. We do not use this task anymore.
*   calculate_mutational_burden (breardon/calculate_mutational_burdenSnapshot ID: 21). This task can be used to calculate the mutational rate of the samples. We do not make use of this data in the release although it could be of interest.
*   summarizeWigFile (breardon/summarizeWigFileSnapshot ID: 5). CCLF ran this task (might be necessary for the mutational burden task). For our workflow, we do not run it.

### Cleaning workspaces

In [ ]:
torm = terra.listHeavyFiles(refworkspace)
h.parrun(['gsutil rm '+i for i in torm], cores=8)
terra.removeFromFailedWorkflows(refworkspace, dryrun=False, everythingFor=['Realign_WES_GATK4','Generate_uBAM_File_List','BamToUnmappedRGBams_MC','CGA_WES_CCLE_ICE','CGA_WES_CCLE_AGILENT'])

## On Terra

In [ ]:
samplesetname

In [ ]:
submission_id1 = refwm.create_submission("CGA_WES_CCLE_ICE", samplesetname, 'sample_set', expression='this.samples')

### Germline

In [ ]:
submission_id2 = refwm.create_submission("cnn_variant_filter_ice", samplesetname, 'sample_set', expression='this.samples')

### copy pairs data to sample data

In [ ]:
terra.waitForSubmission(refworkspace, submission_id1)
pairs = refwm.get_pairs()

In [ ]:
pairs = pairs[pairs.index.isin(tokeep)]
pairs = pairs[~pairs['mutation_validator_validated_maf'].isna()]
pairs = pairs.drop(columns=['case_sample','control_sample','participant_id'])
pairs.index = [i.split('_')[0] for i in pairs.index]

In [ ]:
refwm.update_sample_attributes(pairs)

continuing

In [ ]:
submission_id1 = refwm.create_submission("common_variant_filter", samplesetname, 'sample_set', expression='this.samples')

In [ ]:
#terra.waitForSubmission(refworkspace, submission_id2)
#submission_id2 = refwm.create_submission("aggregate_vcfs", "all")

In [ ]:
terra.waitForSubmission(refworkspace, submission_id1)
submission_id1 = refwm.create_submission("filterMAF_on_CGA_pipeline", samplesetname,'sample_set',expression='this.samples')

### filtered

In [ ]:
terra.waitForSubmission(refworkspace, submission_id1)
submission_id1 = refwm.create_submission("aggregateMAFs_selectFields_filtered", "all")

### unfiltered

In [ ]:
submission_id3 = refwm.create_submission("aggregateMAFs_selectFields_unfiltered", "all")

In [ ]:
terra.waitForSubmission(refworkspace, [submission_id1,submission_id2, submission_id3])

### Save the workflow configurations used

In [ ]:
terra.saveConfigs(refworkspace,'./data/'+samplesetname+'/Mutconfig')

## On local


### Remove some datafile to save money¶

In [ ]:
res = refwm.get_samples()
toremove = ["fixedmate_bam"]
for val in toremove:
    refwm.disable_hound().delete_entity_attributes('sample', res[val], delete_files=True)

In [ ]:
# sometimes it does not work; so better check again
a = res.fixedmate_bam
a = [i for i in a if i is not np.nan]
gcp.rmFiles(a)

### downloading from terra

In [ ]:
sam = refwm.get_samples()

In [ ]:
nowes = set(mutations.DepMap_ID)-set(sam.arxspan_id)
nowes

In [ ]:
nothing = nows -set(ccle_refsamples.arxspan_id)
nothing

### get QC files

In [ ]:
only = samples.index.tolist()

In [ ]:
dataMut = getQC(workspace=refworkspace ,only=only, qcname=["gatk_cnv_all_plots", "lego_plotter_pngs", "copy_number_qc_report", "ffpe_OBF_figures", "mut_legos_html", "oxoG_OBF_figures", "tumor_bam_base_distribution_by_cycle_metrics", "tumor_bam_converted_oxog_metrics"])

In [ ]:
dataBam = getQC(workspace=refworkspace ,only=only, qcname=[ "tumor_bam_alignment_summary_metrics", "tumor_bam_bait_bias_summary_metrics", "tumor_bam_gc_bias_summary_metrics", "tumor_bam_hybrid_selection_metrics", "tumor_bam_insert_size_histogram", "tumor_bam_insert_size_metrics", "tumor_bam_pre_adapter_summary_metrics", "tumor_bam_quality_by_cycle_metrics", "tumor_bam_quality_distribution_metrics", "tumor_bam_quality_yield_metrics"])

In [ ]:
ccle_refsamples = sheets.get(refsheet_url).sheets[0].to_frame(index_col=0)

In [ ]:
for k,v in dataMut.items():
    if k =='nan':
        continue
    curr = ccle_refsamples.loc[k,'processing_qc']
    if curr is np.nan or curr=="":
        curr=set(v)
    else:
        curr = set(curr[1:-1].replace("'","").split(', '))
        curr = set(v) | curr
    ccle_refsamples.loc[k,'processing_qc'] = str(list(curr))
for k,v in dataBam.items():
    if k =='nan':
        continue
    curr = ccle_refsamples.loc[k,'bam_qc']
    if curr is np.nan or curr=="":
        curr=set(v)
    else:
        curr = set(curr[1:-1].replace("'","").split(', '))
        curr = set(v) | curr
    ccle_refsamples.loc[k,'bam_qc'] = str(list(curr))


In [ ]:
dfToSheet(ccle_refsamples,'ccle sample tracker', secret=creds)

In [ ]:
res = refwm.get_sample_sets().loc["all"]
res

### retrieving RNAseq vcfs

In [ ]:
rnamutations = dm.WorkspaceManager(rnaworkspace).get_sample_sets().loc['All_samples']['merged_vcf']
! gsutil cp $rnamutations "temp/rna_mutation_unfiltered_terra_merged.vcf"

### retrieving germline mutations

In [ ]:
snps = res['merged_vcf']
! gsutil cp $snps gs://cclebams/germline_data/wes.all.called.vcf
! gsutil -m acl ch -ru taiga-892@cds-logging.iam.gserviceaccount.com:R gs://cclebams/germline_data/wes.all.called.vcf

### rertrievinng filtered mutations

In [ ]:
filtered = res['filtered_CGA_MAF_aggregated']
! gsutil cp $filtered "temp/mutation_filtered_terra_merged.txt"

In [ ]:
mutations = pd.read_csv('temp/mutation_filtered_terra_merged.txt',sep='\t') 

In [ ]:
mutations = pd.read_csv('temp/mutation_filtered_terra_merged.txt',sep='\t') 
print(mutations.columns[:10])
renaming = removeOlderVersions(names = set(mutations['Tumor_Sample_Barcode']), refsamples = refwm.get_samples(), arxspan_id = "arxspan_id", version="version")
print(len(mutations[mutations['Chromosome']=='0']))

# postprocessing


Here, rather than rerunning the entire analysis, because we know we are adding only WES samples, we can download the previous release's MAF, add the samples, update any annotations, and perform any global filters at the end.

First we need to do an additional step of filtering on coverage and number 

- readMutations
- createSNPs
- addToMainMutation
- filterAllelicFraction
- filterMinCoverage
- mergeAnnotations
- addAnnotation
- maf_add_variant_annotations
- mutation_maf_to_binary_matrix (x3)

In [ ]:
mutations = mutations.rename(columns={"i_ExAC_AF":"ExAC_AF","Tumor_Sample_Barcode":'DepMap_ID',"Tumor_Seq_Allele2":"Tumor_Allele"}).drop(columns=['Center','Tumor_Seq_Allele1'])

In [ ]:
mutations = annotate_likely_immortalized(mutations, TCGAlocs = ['TCGAhsCnt',
'COSMIChsCnt'], max_recurrence=0.05 ,min_tcga_true_cancer=5)

In [ ]:
mutations['CGA_WES_AC'] = [str(i[0]) + ':' + str(i[1]) for i in np.nan_to_num(mutations[['t_alt_count','t_ref_count']].values,0).astype(int)]

In [ ]:
mutations = filterCoverage(mutations, loc=['CGA_WES_AC'], sep=':',cov=2)

In [ ]:
mutations = filterAllelicFraction(mutations, loc=['CGA_WES_AC'], sep=':',frac=0.1)

In [ ]:
#Count the total number of mutations per cell line, split by type (SNP, INS, DEL)
#Count the total number of mutations observed by position

In [ ]:
mutations = addAnnotation(mutations, NCBI_Build='37', Strand="+")

In [ ]:
mutations.to_csv('temp/wes_somatic_mutations_withduplicates_'+samplesetname+'.csv', index=False)

In [ ]:
mutations= pd.read_csv('temp/wes_somatic_mutations_withduplicates_'+samplesetname+'.csv')

In [ ]:
mutations

In [ ]:
tokeep = {"CDS-mys9Dm":"ACH-001955","CDS-Rl87Z1":"ACH-001956","CDS-TzQAjG":"ACH-001957"}

In [ ]:
# based on QC from the CN pipeline: can't be done in mut because not a quantitativ pipeline
%store -r wesfailed
%store -r wesdup
%store -r renaming
%store -r issues
#removing dups from replicates
mutations = mutations[~mutations.DepMap_ID.isin((set(wesdup[:,1])|set(wesfailed)) - set(tokeep.keys()))]

In [ ]:
renaming.update(tokeep)
renaming.update({"CDS-Ckptje": "ACH-001672",
"CDS-pgDmZb": "ACH-002291"})

In [ ]:
priomutations = mutations[mutations.DepMap_ID.isin(renaming.keys())].replace(renaming)

### Adding WGS's exonic mutation

In [ ]:
wgsmutations = pd.read_csv('temp/wgs_somatic_mutations_'+samplesetname+'.csv')

In [ ]:
#for now we keep WES if we have them
toadd = set(wgsmutations.DepMap_ID) - set(mutations.DepMap_ID)
toadd

In [ ]:
priomutations = priomutations.append(wgsmutations[wgsemutations.DepMap_ID.isin(toadd)]).reset_index(drop=True)

In [ ]:
ccle_refsamples = sheets.get(refsheet_url).sheets[0].to_frame(index_col=0)

In [ ]:
normals = set(ccle_refsamples[ccle_refsamples.primary_disease=="normal"].arxspan_id)
wrong = set(['ACH-001189', 'ACH-002303', 'ACH-002315', 'ACH-002341', 'ACH-001011', 'ACH-001108', 'ACH-001187','ACH-003000', 'ACH-002875', 'ACH-002874'])

In [ ]:
priomutations = priomutations[~priomutations.DepMap_ID.isin(wrong)]

In [ ]:
#mutations = mutations[mutations.DepMap_ID.isin(normals)]

In [ ]:
priomutations.to_csv('temp/wes_somatic_mutations_all_'+samplesetname+'.csv', index=False)

In [ ]:
priomutations = pd.read_csv('temp/wes_somatic_mutations_all_'+samplesetname+'.csv')

In [ ]:
priomutations = priomutations.reset_index(drop=True)

In [ ]:
mafToMat(priomutations[(priomutations.isDeleterious) & (priomutations['tumor_f']>0.25)]).T.to_csv('temp/wes_somatic_mutations_deleterious_matrix.csv')
mafToMat(priomutations[~(priomutations.isDeleterious | priomutations.isCOSMIChotspot | priomutations.isTCGAhotspot | priomutations['Variant_Classification']=='Silent') & (priomutations['tumor_f']>0.25)]).T.to_csv('temp/wes_somatic_mutations_other_matrix.csv')
mafToMat(priomutations[(priomutations.isCOSMIChotspot | priomutations.isTCGAhotspot) &(priomutations['tumor_f']>0.25)]).T.to_csv('temp/wes_somatic_mutations_hotspot_matrix.csv')

In [ ]:
mafToMat(priomutations[(priomutations.isDeleterious) & (priomutations['tumor_f']>0.25)], boolify=True).astype(int).T.to_csv('temp/wes_somatic_mutations_deleterious_boolmatrix.csv')
mafToMat(priomutations[~(priomutations.isDeleterious | priomutations.isCOSMIChotspot | priomutations.isTCGAhotspot | priomutations['Variant_Classification']=='Silent') & (priomutations['tumor_f']>0.25)], boolify=True).astype(int).T.to_csv('temp/wes_somatic_mutations_other_boolmatrix.csv')
mafToMat(priomutations[(priomutations.isCOSMIChotspot | priomutations.isTCGAhotspot) & (priomutations['tumor_f']>0.25)], boolify=True).astype(int).T.to_csv('temp/wes_somatic_mutations_hotspot_boolmatrix.csv')

In [ ]:
legacy_hybridcapture = tc.get(name='mutations-da6a', file='legacy_hybridcapture_somatic_mutations').drop(columns=['Unnamed: 0',"Tumor_Sample_Barcode"]).rename(columns={'Tumor_Seq_Allele1':'Tumor_Allele'})
legacy_raindance = tc.get(name='mutations-da6a', file='legacy_raindance_somatic_mutations').drop(columns=['Unnamed: 0',"Tumor_Sample_Barcode"]).rename(columns={'Tumor_Seq_Allele1':'Tumor_Allele'})
legacy_rna = tc.get(name='mutations-da6a', file='legacy_rna_somatic_mutations').drop(columns=['Unnamed: 0',"Tumor_Sample_Barcode"]).rename(columns={'Tumor_Seq_Allele1':'Tumor_Allele'})
legacy_wes_sanger = tc.get(name='mutations-da6a', file='legacy_wes_sanger_somatic_mutations').drop(columns=['Unnamed: 0',"Tumor_Sample_Barcode"]).rename(columns={'Tumor_Seq_Allele1':'Tumor_Allele'})
legacy_wgs_exoniconly = tc.get(name='mutations-da6a', file='legacy_wgs_exoniconly_somatic_mutations').drop(columns=['Unnamed: 0',"Tumor_Sample_Barcode"]).rename(columns={'Tumor_Seq_Allele1':'Tumor_Allele'})

solving issues with the legacy datasets

In [ ]:
legacy_hybridcapture.loc[legacy_hybridcapture[legacy_hybridcapture['Variant_Classification'].isna()].index,'Variant_Classification']='Missense_Mutation'

In [ ]:
legacy_wgs_exoniconly.loc[legacy_wgs_exoniconly[legacy_wgs_exoniconly['Genome_Change'].isna()].index, 'Genome_Change'] = ['g.chr'+str(i.Chromosome)+":"+str(i.Start_position)+i.Reference_Allele+">"+i.Tumor_Allele for _, i in legacy_wgs_exoniconly[legacy_wgs_exoniconly['Genome_Change'].isna()].iterrows()]

In [ ]:
legacy_wes_sanger.loc[legacy_wes_sanger[legacy_wes_sanger['Genome_Change'].isna()].index, 'Genome_Change'] = ['g.chr'+str(i.Chromosome)+":"+str(i.Start_position)+i.Reference_Allele+">"+i.Tumor_Allele for _, i in legacy_wes_sanger[legacy_wes_sanger['Genome_Change'].isna()].iterrows()]

In [ ]:
legacy_wes_sanger[legacy_wes_sanger.DepMap_ID=="ACH-002378"]

In [ ]:
legacy_raindance.loc[legacy_raindance[legacy_raindance['Genome_Change'].isna()].index, 'Genome_Change'] = ['g.chr'+str(i.Chromosome)+":"+str(i.Start_position)+i.Reference_Allele+">"+i.Tumor_Allele for _, i in legacy_raindance[legacy_raindance['Genome_Change'].isna()].iterrows()]

In [ ]:
legacy_hybridcapture.loc[legacy_hybridcapture[legacy_hybridcapture['Genome_Change'].isna()].index, 'Genome_Change'] = ['g.chr'+str(i.Chromosome)+":"+str(i.Start_position)+i.Reference_Allele+">"+i.Tumor_Allele for _, i in legacy_hybridcapture[legacy_hybridcapture['Genome_Change'].isna()].iterrows()]

In [ ]:
legacy_rna.loc[legacy_rna[legacy_rna['Genome_Change'].isna()].index, 'Genome_Change'] = ['g.chr'+str(i.Chromosome)+":"+str(i.Start_position)+i.Reference_Allele+">"+i.Tumor_Allele for _, i in legacy_rna[legacy_rna['Genome_Change'].isna()].iterrows()]

In [ ]:
todrop = []
legacy_rna['loci'] = legacy_rna['DepMap_ID']+"_"+legacy_rna['Chromosome']+"_"+legacy_rna['Start_position'].astype(str)
for val in h.dups(legacy_rna.loci):
    todrop.append(legacy_rna[legacy_rna.loci==val].index[0])
legacy_rna = legacy_rna.drop(todrop)

In [ ]:
#legacy_hybridcapture = legacy_hybridcapture[~legacy_hybridcapture.DepMap_ID.isin(normals)]
#legacy_raindance = legacy_raindance[~legacy_raindance.DepMap_ID.isin(normals)]
#legacy_wes_sanger = legacy_wes_sanger[~legacy_wes_sanger.DepMap_ID.isin(normals)]

In [ ]:
legacy_wgs_exoniconly = legacy_wgs_exoniconly[~legacy_wgs_exoniconly.DepMap_ID.isin(wrong)]
legacy_wes_sanger = legacy_wes_sanger[~legacy_wes_sanger.DepMap_ID.isin(wrong)]
legacy_raindance = legacy_raindance[~legacy_raindance.DepMap_ID.isin(wrong)]
legacy_rna = legacy_rna[~legacy_rna.DepMap_ID.isin(wrong)]
legacy_hybridcapture = legacy_hybridcapture[~legacy_hybridcapture.DepMap_ID.isin(set(wrong)-set(["ACH-001187", "ACH-001011", "ACH-001108"]))]

In [ ]:
merged = mergeAnnotations(priomutations, legacy_hybridcapture, useSecondForConflict=True, dry_run=False)
merged = mergeAnnotations(merged, legacy_raindance, useSecondForConflict=True, dry_run=False)
merged = mergeAnnotations(merged, legacy_wgs_exoniconly, useSecondForConflict=False, dry_run=False)
merged = mergeAnnotations(merged, legacy_wes_sanger, useSecondForConflict=False, dry_run=False)
merged = mergeAnnotations(merged, legacy_rna, useSecondForConflict=False, dry_run=False)

In [ ]:
mutation_groups={
"other conserving": ["5'Flank", "Intron", "IGR", "3'UTR", "5'UTR"],
"other non-conserving":["In_Frame_Del", "In_Frame_Ins", "Stop_Codon_Del", "Stop_Codon_Ins", "Missense_Mutation", "Nonstop_Mutation"],
'silent': ['Silent'],
"damaging":['De_novo_Start_OutOfFrame','Frame_Shift_Del','Frame_Shift_Ins', 'Splice_Site', 'Start_Codon_Del', 'Start_Codon_Ins', 'Start_Codon_SNP','Nonsense_Mutation']
}

In [ ]:
rename = {}
for k,v in mutation_groups.items():
    for e in v:
        rename[e] = k
merged['Variant_annotation'] = [rename[i] for i in merged['Variant_Classification'].tolist()]

### Compare to previous release

I would run some checks here comparing the results to the previous releases MAF. Namely:

- Count the total number of mutations per cell line, split by type (SNP, INS, DEL)
- Count the total number of mutations observed by position (group by chromosome, start position, end position and count the number of mutations)
- Look at specific differences between the two MAFs (join on DepMap_ID, Chromosome, Start position, End position, Variant_Type). I would do this for WES only

### check important mutations

In [ ]:
# check MOLM13, MV411 cell lines- The well known mutation status of FLT3

In [ ]:
# check TP53 mutation 

Are mutation consistent?

QC mutations, for a known dependency, check if it matches mutation of this gene. (if P53 is mutated, cannot have dependency on P53 or MDM2 MDM4/ inverse fir BRAF and KRAF to themselves)

### saving this version

In [ ]:
a = set(merged.DepMap_ID) 
b = set(tc.get(name='depmap-a0ab', file='CCLE_mutations').DepMap_ID)
print("new lines:")
print(a-b)
print('lost lines:')
print(b-a)

In [ ]:
merged[merged.DepMap_ID=="ACH-001194"]

In [ ]:
merged.to_csv('temp/wes_somatic_mutations_withlegacy_'+samplesetname+'.csv', index=False)

In [ ]:
merged = pd.read_csv('temp/wes_somatic_mutations_withlegacy_'+samplesetname+'.csv')

In [ ]:
mafToMat(merged[merged.Variant_annotation=="damaging"], boolify=True).astype(int).T.to_csv('temp/all_somatic_mutations_boolmatrix_fordepmap_damaging.csv')
mafToMat(merged[merged.Variant_annotation=="other conserving"], boolify=True).astype(int).T.to_csv('temp/all_somatic_mutations_boolmatrix_fordepmap_othercons.csv')
mafToMat(merged[merged.Variant_annotation=="other non-conserving"], boolify=True).astype(int).T.to_csv('temp/all_somatic_mutations_boolmatrix_fordepmap_othernoncons.csv')
mafToMat(merged[(merged.isCOSMIChotspot | merged.isTCGAhotspot)], boolify=True).astype(int).T.to_csv('temp/all_somatic_mutations_boolmatrix_fordepmap_hotspot.csv')

### saving samples used for this release

should be the same as in CN otherwise need to do something more complex

In [ ]:
#ccle_refsamples.loc[renaming.keys(),samplesetname]=1
#ccle_refsamples.loc[ccle_refsamples[ccle_refsamples.arxspan_id.isin(toadd) & ccle_refsamples.datatype=="wgs"].index,samplesetname]=1
#ccle_refsamples.to_csv('temp/newrefWES.csv')

## retrieving unfiltered mutations

In [ ]:
unfiltered = res['unfiltered_CGA_MAF_aggregated']
! gsutil cp $unfiltered "temp/wes_mutation_unfiltered_terra_merged.txt"

In [ ]:
unfiltered = pd.read_csv('temp/wes_mutation_unfiltered_terra_merged.txt', sep='\t', encoding='L6',na_values=["__UNKNOWN__",'.'], engine='c', dtype=str)

In [ ]:
unfiltered['somatic'] = unfiltered['somatic'].replace('nan','False')
unfiltered['HGNC_Status'] = unfiltered['HGNC_Status'].replace('nan','Unapproved')
unfiltered['judgement'] = unfiltered['judgement'].replace('nan','REMOVE')
unfiltered = unfiltered.rename(columns={"i_ExAC_AF":"ExAC_AF","Tumor_Sample_Barcode":'DepMap_ID',"Tumor_Seq_Allele2":"Tumor_Allele"}).drop(columns=['Tumor_Seq_Allele1'])
unfiltered['CGA_WES_AC'] = [str(i[0]) + ':' + str(i[1]) for i in np.nan_to_num(unfiltered[['t_alt_count','t_ref_count']].values.astype(float),0).astype(int)]

In [ ]:
toremove = []
subunfilt = unfiltered.iloc[:10000]
for i, val in enumerate(unfiltered.columns):
    h.showcount(i,len(unfiltered.columns))
    if len(set(subunfilt[val])-set(['nan']))==1:
        if len(set(unfiltered[val])-set(['nan']))==1:
            toremove.append(val)
unfiltered = unfiltered.drop(columns=set(toremove))

In [ ]:
toint =  ["Start_position", "End_position"]
for val in toint:
    unfiltered[val]  = unfiltered[val].astype(int)

In [ ]:
unfiltered.to_csv('temp/mutation_somatic_unfiltered_withreplicates.csv.gz', index=False)

In [ ]:
unfiltered = unfiltered[unfiltered.DepMap_ID.isin(renaming.keys())]
unfiltered['DepMap_ID'] = unfiltered['DepMap_ID'].replace(renaming)

In [ ]:
unfiltered.to_csv('temp/mutation_somatic_unfiltered_all.csv.gz', index=False)

In [ ]:
del unfiltered

# uploading on taiga

## Saving to latest version

In [ ]:
#!gunzip temp/wes_mutation_somatic_unfiltered_withreplicates.csv.gz
!gunzip temp/wes_mutation_somatic_unfiltered_all.csv.gz

In [ ]:
tc.update_dataset(dataset_permaname="mutations-latest-ed72",
                 upload_file_path_dict={
        'temp/wes_somatic_mutations_withduplicates_'+samplesetname+'.csv': 'TableCSV',
        'temp/wgs_somatic_mutations_'+samplesetname+'.csv': 'TableCSV',
        'temp/wes_somatic_mutations_deleterious_matrix.csv': 'NumericMatrixCSV',
        'temp/wes_somatic_mutations_other_matrix.csv': 'NumericMatrixCSV',
        'temp/wes_somatic_mutations_hotspot_matrix.csv': 'NumericMatrixCSV',
        'temp/wes_somatic_mutations_all_'+samplesetname+'.csv': 'TableCSV',
        'temp/wes_somatic_mutations_deleterious_boolmatrix.csv': 'NumericMatrixCSV',
        'temp/wes_somatic_mutations_other_boolmatrix.csv': 'NumericMatrixCSV',
        'temp/wes_somatic_mutations_hotspot_boolmatrix.csv': 'NumericMatrixCSV',
        'temp/wes_mutation_somatic_unfiltered_all.csv': 'TableCSV',
        'temp/all_somatic_mutations_boolmatrix_fordepmap_damaging.csv': 'NumericMatrixCSV',
        'temp/all_somatic_mutations_boolmatrix_fordepmap_othercons.csv': 'NumericMatrixCSV',
        'temp/all_somatic_mutations_boolmatrix_fordepmap_othernoncons.csv': 'NumericMatrixCSV',
        'temp/all_somatic_mutations_boolmatrix_fordepmap_hotspot.csv': 'NumericMatrixCSV',
        #'temp/wes_mutation_somatic_unfiltered_withreplicates.csv': 'TableCSV',
        }, 
                 dataset_description="""
# Mutations

PORTAL TEAM SHOULD NOT USE THIS: There are lines here that should not make it even to internal.

/!\ This is the most up to date version of the CCLE Mutatios data.
The data is most likely of a better quality that what is on other folder. It is however in beta version as not all changes have either been confirmed or accepted by the DepMap Ops and the DepMap Portal Team.

# Notations:

all: every cell lines we have

WES: all data comes from the WExomeS samples we posses

WGS: all data comes from the WGenomeS samples we posses

withreplicates: if we have two different sequencing from a sample, we kept both, see the depmap sample tracker for annotations [https://docs.google.com/spreadsheets/d/1XkZypRuOEXzNLxVk9EOHeWRE98Z8_DBvL4PovyM01FE](https://docs.google.com/spreadsheets/d/1XkZypRuOEXzNLxVk9EOHeWRE98Z8_DBvL4PovyM01FE). this dataset is more geared toward QC or in-depth analysis of a particular cell line.

merged: everything from both WGS and WES

latest: only the latest sequencing versions of the samples were kept

genes (gene rpkm):
__Rows__:
__Columns__:
Counts (gene counts):
__Rows__:
__Columns__:
Gene level CN data:
__Rows__:
__Columns__:
 DepMap cell line IDs
 gene names in the format HGNC\_symbol (Entrez\_ID)
DepMap\_ID, Chromosome, Start, End, Num\_Probes, Segment\_Mean
 """)